In [107]:
import math
import re
from collections import Counter

import datetime
import pandas as pd
import ast

In [4]:
WORD = re.compile(r"\w+")

# get current year to save in applicable folder
current_year = datetime.date.today().year

In [5]:
# data import
journal_ratings_scimago = pd.read_csv(f'../data/{current_year}/journal_ratings_scimago.csv', sep=";")
journal_ratings_vhb = pd.read_csv(f'../data/{current_year}/journal_ratings_vhb.csv')

publications_test_gs = pd.read_csv(f'../data/{current_year}/publications_test_gs.csv')
publications_test_rg = pd.read_csv(f'../data/{current_year}/publications_test_rg.csv')

C:\Users\katma\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# functions to get similarity of strings using cosine similarity

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [70]:
def is_similar(text1, text2):
    
    vector1 = text_to_vector(text1.lower())
    vector2 = text_to_vector(text2.lower())

    cosine = get_cosine(vector1, vector2)
    print(cosine)
    
    if cosine > 0.7:
        similar = True
    else:
        similar = False
        
    return similar

In [79]:
def get_not_none_string(string1, string2):
    
    if string1 and not string2:
        return string1
    
    elif string2 and not string1:
        return string2
    
    elif string1 and string2:
        return max([string1, string2], key=len)
    
    else:
        return None

In [116]:
def combine_publication(row_gs, row_rg):
    
    title = get_not_none_string(row_gs['PUB_TITLE'], row_rg['PUB_TITLE'])
    year = max([row_gs['PUB_YEAR'], row_rg['PUB_YEAR']])
    publisher = get_not_none_string(row_gs['PUB_PUBLISHER'], row_rg['PUB_PUBLISHER'])
    
    authors_gs = [n.strip() for n in ast.literal_eval(row_gs["PUB_AUTHORS"])]
    authors_rg = [n.strip() for n in ast.literal_eval(row_rg["PUB_AUTHORS"])]
    authors = list(set(authors_gs + authors_rg))

    
    data = {
        'AUTHOR_NAME': row_gs["AUTHOR_NAME"],
        'PUB_TITLE': title, 
        'PUB_YEAR': year, 
        'PUB_AUTHORS': authors,  
        'PUB_TYPE': row_rg['PUB_TYPE'],
        'PUB_PUBLISHER': publisher,
        'PUB_ISBN': row_rg['PUB_ISBN'],
        'PUB_DOI': row_rg['PUB_DOI']
       }

    return data

In [28]:
# get all unique names
all_author_names = publications_test_gs['AUTHOR_NAME'].tolist() + publications_test_rg['AUTHOR_NAME'].tolist()
all_author_names = set(all_names) # make unique

In [119]:
#for author_name in all_author_names:
author_name = "Stephan Daurer" 
    
subset_test_gs = publications_test_gs[publications_test_gs['AUTHOR_NAME'] == author_name]
subset_test_rg = publications_test_rg[publications_test_rg['AUTHOR_NAME'] == author_name]

In [118]:
pd_result = pd.DataFrame(columns=['AUTHOR_NAME', 'PUB_TITLE', 'PUB_YEAR', 
                                                  'PUB_AUTHORS', 'PUB_TYPE', 'PUB_PUBLISHER', 
                                                  'PUB_ISBN', 'PUB_DOI' ])

has_publications = True
while(has_publications):
    if len(subset_test_gs) == 0:
        has_publications = False
        
    else:
        # get title of first GS publication
        title_gs = subset_test_gs.iloc[0]["PUB_TITLE"]
        duplicate_found = False
        
        # iterate over RG subset
        for index, row in subset_test_rg.iterrows():
            if is_similar(row["PUB_TITLE"], title_gs): # similar title found
                duplicate_found = True
                print(row["PUB_TITLE"])
                print(title_gs)
            
                # remove the found match from the subsets
                subset_test_gs = subset_test_gs.drop(index=0).reset_index(drop=True)
                subset_test_rg = subset_test_rg.drop(index).reset_index(drop=True)
                
                combined_row = combine_publication(subset_test_gs.iloc[0], row)
                print(combined_row)
                pd_result.append(combined_row , ignore_index=True)
            
        if not duplicate_found:
            print(title_gs)
            pd_result.append(subset_test_gs.iloc[0], ignore_index=True)
            subset_test_gs = subset_test_gs.drop(index=0).reset_index(drop=True)
            
    

0.857142857142857
Einführung in die Wirtschaftsinformatik: Ein fallstudienbasiertes Lehrbuch
Einführung in die Wirtschaftsinformatik: Ein fallstudienbasierendes Lehrbuch
{'AUTHOR_NAME': 'Stephan Daurer', 'PUB_TITLE': 'Digitalisierung und Konvergenz von Online-und Offline-Welt: Einfluss der mobilen Internetsuche auf das Kaufverhalten', 'PUB_YEAR': 2021.0, 'PUB_AUTHORS': ['S Daurer', 'Arthur Kolb', 'Michael A. Bächle', 'M Spann', 'D Molitor', 'Stephan Daurer'], 'PUB_TYPE': 'Book', 'PUB_PUBLISHER': 'Zeitschrift für Betriebswirtschaft (ZfB)', 'PUB_ISBN': ' 9783110722253', 'PUB_DOI': '10.1515/9783110722260'}
0.0
0.0
0.14285714285714285
0.08671099695241198
0.857142857142857
Einführung in die Wirtschaftsinformatik: Ein fallstudienbasiertes Lehrbuch
Einführung in die Wirtschaftsinformatik: Ein fallstudienbasierendes Lehrbuch
{'AUTHOR_NAME': 'Stephan Daurer', 'PUB_TITLE': 'Einführung in die Wirtschaftsinformatik: Ein fallstudienbasiertes Lehrbuch', 'PUB_YEAR': 2018.0, 'PUB_AUTHORS': ['S Daurer'

TypeError: object of type 'float' has no len()

In [115]:
pd_result

,AUTHOR_NAME,PUB_TITLE,PUB_YEAR,PUB_AUTHORS,PUB_TYPE,PUB_PUBLISHER,PUB_ISBN,PUB_DOI
